In [ ]:
num_sample = 300
num_burn = 20
sample_size = 800
n_cpu = 10

min_degree = 1
max_degree = 2

In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.01,0,0,0.01,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0.20])     

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

1 2


100%|██████████| 320/320 [00:00<00:00, 386.43it/s]


(-0.13436200009866905, 0.4307375)

In [3]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

100%|██████████| 200/200 [00:00<00:00, 589.83it/s]


{'average': 0.3026875,
 'direct_effect': -0.1387500000000001,
 'spillover_effect': -0.0729499999999999,
 'psi_1_gamma': 0.26037499999999997,
 'psi_0_gamma': 0.39912500000000006,
 'psi_zero': 0.47207499999999997}

In [4]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


Multiprocessing <function run_autognet at 0x12c128b80> in 100 tasks, with 10 processes...


100%|██████████| 60/60 [00:00<00:00, 523.90it/s]


Multiprocessing finished.


In [5]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

average: 0.30567 ± 0.02158
direct_effect: -0.12902 ± 0.03214
spillover_effect: -0.06798 ± 0.03638
psi_1_gamma: 0.26585 ± 0.02511
psi_0_gamma: 0.39487 ± 0.02890
psi_zero: 0.46285 ± 0.03225


In [6]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")

Multiprocessing <function run_dr at 0x1052a0700> in 100 tasks, with 10 processes...
Multiprocessing finished.
average: 0.28467 ± 0.02993
direct_effect: -0.13771 ± 0.04828
spillover_effect: -0.06842 ± 0.04645
psi_1_gamma: 0.25642 ± 0.03674
psi_0_gamma: 0.39413 ± 0.02948
psi_zero: 0.46255 ± 0.03842


average: 0.27609 ± 0.03565
direct_effect: -0.11926 ± 0.06451
spillover_effect: -0.08767 ± 0.05838
psi_1_gamma: 0.25561 ± 0.04321
psi_0_gamma: 0.37487 ± 0.04515
psi_zero: 0.46255 ± 0.03842

In [ ]:
from dgp import sample_network_chain, get_graph, sample_Y1, sample_Y2, agcEffect
import numpy as np

# 1. Simulate or load data
adj = get_graph(sample_size, min_degree, max_degree, seed=1)
print(adj.sum(axis=1).min(), adj.sum(axis=1).max())

tau = np.array([-1.0, 0.50, -0.50])       # shape (3,)
rho = np.array([[0,0.1,0.2],
                [0.1,0,0.1],
                [0.2,0.1,0]])      # shape (3, 3), with 0s on the diagonal
nu = np.array([0.1,0,0,0.01,0,0,0.01,0,0]).reshape(3,3)       # shape (3, 3)
gamma = np.array([-1.00,0.50,0.10,0.20,0.05,0.25,-0.08,0.30])     # shape (8,)
beta = np.array([-0.30,-0.60,-0.20,-0.20,-0.05,-0.10,-0.01,0.40,0.01,0])     

Y_chain, A_chain, L_chain = sample_network_chain(adj, tau, rho, nu, gamma, beta, R=num_sample,
    burnin_R=num_burn, seed=0, sample_Y_func=sample_Y1, Atype=('gen', 0.7))

Y_chain = Y_chain[::3]
A_chain = A_chain[::3]
L_chain = L_chain[::3]

dir, dir2 = [], []
for i in range(Y_chain.shape[0]):
    Y = Y_chain[i]
    A = A_chain[i]
    L = L_chain[i]
    dir.append(np.mean(Y[A==1]) - np.mean(Y[A==0]))
    dir2.append(np.mean(A))

np.mean(dir), np.mean(dir2)

In [ ]:
agcEffect(
    adj,
    tau, rho, nu, beta,
    treatment_allocation=0.7,
    R=100,
    burnin_R=100,
    seed=0,
    sample_Y_func=sample_Y1
)

In [ ]:
from utils import run_pll
from run_pll import run_autognet

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_ag = run_pll(run_autognet, args, processes=n_cpu)
res_list_array_ag = np.array(res_list_ag)


In [ ]:
from run_pll import column_names

ret_mean_ag = res_list_array_ag.mean(axis=0)
ret_std_ag = res_list_array_ag.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_ag[i]:.5f} ± {ret_std_ag[i]:.5f}")

In [ ]:
from run_pll import *
from utils import run_pll

args = [{'Y_chain': Y_chain,
         'A_chain': A_chain,
         'L_chain': L_chain,
         'adj': adj,
         'i': i} for i in range(len(Y_chain))]

res_list_dr = run_pll(run_dr, args, processes=n_cpu)
res_list_array_dr = np.array(res_list_dr)

from run_pll import column_names

ret_mean_dr = res_list_array_dr.mean(axis=0)
ret_std_dr = res_list_array_dr.std(axis=0)
for i in range(len(column_names)):
    print(f"{column_names[i]}: {ret_mean_dr[i]:.5f} ± {ret_std_dr[i]:.5f}")